# Earthquake seismology with DAS

#### Todo List:
1. Reference earthquakes by Evid rather than dataframe index
2. Make a map of all of the detected events
3. Try some xcorr

In [1]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import numpy as np
import datetime
import pandas as pd
from dasquakes import sintela_to_datetime, open_sintela_file, local_earthquake_quicklook
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from libcomcat.search import search
from libcomcat.dataframes import get_summary_data_frame

ERROR 1: PROJ: proj_create_from_database: Open of /home/velgueta/.conda/envs/python-obspy/share/proj failed


### Get a list of events from USGS

In [2]:
events = search(starttime = datetime.datetime(2022, 3, 17, 0, 0), 
                endtime   = datetime.datetime.now(),
                minmagnitude=3.0,
                maxradiuskm=600, 
                latitude=47.60, 
                longitude=-122.33)

print("Returned %s events" % len(events))
 

Returned 27 events


### Choose an event to analyze

In [3]:
event_df = get_summary_data_frame(events)
event_df = event_df.sort_values(by=['time'],ascending=False)
event_df

#event_df.iloc[0::]

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
26,us7000j1kj,2023-01-02 08:49:35.846,"228 km W of Bandon, Oregon",43.428900,-127.190900,10.000,3.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,234
25,us7000ishe,2022-11-26 03:50:17.627,"Vancouver Island, Canada region",49.276600,-126.099400,34.283,4.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,527
24,us7000iqsk,2022-11-18 15:42:45.032,"183 km W of Bandon, Oregon",43.374100,-126.635600,10.000,4.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,298
23,us7000imha,2022-11-04 14:16:58.043,Off the coast of Oregon,43.401300,-127.099500,10.000,3.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,178
22,us7000im59,2022-11-03 09:57:33.841,Off the coast of Oregon,43.327300,-126.840700,10.000,3.50,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,189
21,us7000ikhh,2022-10-26 15:18:19.752,"169 km W of Langlois, Oregon",43.173500,-126.500500,10.000,3.20,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,158
20,us6000itsl,2022-10-14 21:23:02.037,"152 km WNW of Port Orford, Oregon",43.070100,-126.307200,10.000,3.30,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,168
19,uw61886506,2022-10-07 12:52:36.010,"16 km ESE of Lacomb, Oregon",44.540333,-122.550833,13.170,4.39,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,896
18,us6000ipd3,2022-09-29 15:09:32.565,off the coast of Washington,47.637400,-128.921800,10.555,3.30,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,168
17,uw61882431,2022-09-22 14:41:56.900,"26 km WNW of Forks, Washington",48.046500,-124.715000,37.040,3.49,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,190


In [4]:
event_df.iterrows()

<generator object DataFrame.iterrows at 0x7fdc60d950b0>

In [6]:
for index,this_event in event_df.iterrows():
    print(this_event['id'])
    this_id = this_event['id']

    this_event_new = event_df[event_df.id==this_id]
    stitle=f"M {this_event_new['magnitude'].to_numpy()[0]}, "\
           f"{this_event_new['location'].to_numpy()[0]}, "\
           f"{str(this_event_new['time'].to_numpy()[0])[0:-10]} UTC"
      

        
    cable = 'whidbey'
    record_length = 10 #minutes
    t0 = this_event_new['time'].iloc[0]
        
    prefix = 'whidbey'
    network_name='Whidbey-DAS'
    datastore = '/data/data5/Converted/'
    try:
        data,dates,attrs = open_sintela_file(prefix,
                                     t0,
                                     datastore,
                                     number_of_files=record_length,
                                      verbose=True)
        x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
        low_cut = 2
        hi_cut = 10

        b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
        data_filt = filtfilt(b,a,data,axis=0)
        
        filename = f"earthquakes_figures_600kmradius/"+this_id
        local_earthquake_quicklook(dates,data_filt,0,0,x_max,
                           stitle,filename=filename,das_vmax=0.02,
                           network_name=network_name,
                           skip_seismograms=True)

        
    except ValueError:
        continue
        # x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
   # low_cut = 2
    #hi_cut = 10

    #b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
    #data_filt = filtfilt(b,a,data,axis=0)
        
    #filename = f"earthquakes_figures"
    #local_earthquake_quicklook(dates,data_filt,x_max,
                      #     stitle,filename=filename,das_vmax=0.02,
                       #    network_name=network_name,
                        #   skip_seismograms=True)


us7000j1kj
Searching for files matching: /data/data5/Converted/whidbey_2023-01-02_08-49*h5
us7000ishe
Searching for files matching: /data/data5/Converted/whidbey_2022-11-26_03-50*h5
us7000iqsk
Searching for files matching: /data/data5/Converted/whidbey_2022-11-18_15-42*h5
us7000imha
Searching for files matching: /data/data5/Converted/whidbey_2022-11-04_14-16*h5
us7000im59
Searching for files matching: /data/data5/Converted/whidbey_2022-11-03_09-57*h5
us7000ikhh
Searching for files matching: /data/data5/Converted/whidbey_2022-10-26_15-18*h5
us6000itsl
Searching for files matching: /data/data5/Converted/whidbey_2022-10-14_21-23*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-10-14_21-24*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-10-14_21-25*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-10-14_21-26*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-10-14_21-27*h5
Searching for files matching: /data/data5/C

Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_20-05*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_20-06*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_20-07*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_20-08*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_20-09*h5
us6000iabx
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_19-14*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_19-15*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_19-16*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_19-17*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_19-18*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_19-19*h5
Searching for files matching: /data/data5/Converted/whidbey_2022-08-12_19-20*h5
Searching for files matching:

In [ ]:
#detected_quakes = ['uw61835766','uw61832097','uw61836427',
                 #  'uw61846212','uw61819867','uw61850651','uw61855012']
#detected_quakes_whidbey = ['nc73821036','uw61819867','uw61855012','us7000gxl1']

#this_event_index = 0
#this_id=detected_quakes[this_event_index]

this_id = 'uw61832097'

this_event = event_df[event_df.id==this_id]
t0 = this_event['time'].iloc[0]

this_event

In [ ]:
print(t0)

In [ ]:
# save each event in format cvs and hdf5
#make statistics with the amplitud of each event-compare the amplitud